In [9]:
import pandas as pd
import PyPDF2
import fitz
from io import StringIO
import shutil
import os

In [10]:
# This script takes the same annotations in one document of reference

doc = fitz.open('document_with_annotations.pdf')

rect_lists = [[] for i in range(4)] 

for page in doc:
    annots = page.annots()
    for i, annot in enumerate(annots):
        rects = annot.rect
        for rect in rects:
            rect_lists[i].append(rect)

#for rect_list in rect_lists:
    #print(rect_list)


In [11]:
# Here we iterate every document in one folder, draw the annotations and save with different name in another folder

SOURCE_FOLDER = 'SOURCE_FOLDER'
DEST_FOLDER = 'DEST_FOLDER'

for filename in os.listdir(SOURCE_FOLDER):
    doc2 = fitz.open(os.path.join(SOURCE_FOLDER, filename))
    page = doc2[0]
    for rect_list in rect_lists:    
        page.add_rect_annot(rect_list)
    doc2.save(os.path.join(DEST_FOLDER, 'CAPT_' + filename))
    doc2.close()

In [12]:
def make_text(words):
    line_dict = {} 
    words.sort(key=lambda w: w[0])
    for w in words:  
        y1 = round(w[3], 1)  
        word = w[4] 
        line = line_dict.get(y1, [])  
        line.append(word)  
        line_dict[y1] = line  
    lines = list(line_dict.items())
    lines.sort()  
    return "n".join([" ".join(line[1]) for line in lines])

In [22]:
all_annots = []

for filename in os.listdir(DEST_FOLDER):
    doc2 = fitz.open(os.path.join(DEST_FOLDER, filename))
    page = doc2[0]
    for annot in page.annots():
        if annot!=None:
            rec = annot.rect
            words = page.get_text("words") 
            mywords = [w for w in words if fitz.Rect(w[:4]) in rec]
            ann = make_text(mywords)
            all_annots.append(ann)

In [23]:
print(all_annots)

['CORNELIO URREA MONTES', 'UEMC730326XXX', '150202988300', 'Contado', 'JOSE ALFONSO TOLEDO LICEA', 'TOLA840813XXX', '150203048000', 'Contado', 'PABLO ANTONIO HERRERA', 'HECP911101XXX', '150203224600', 'Contado']


In [4]:
# On this script we can eliminate the original files after saving into the new folder 

for root, dirs, files in os.walk(DEST_FOLDER):
    for file in files:
        if file.startswith("CAPT_"):
            path = os.path.join(root, file)
            file_name = file.replace("CAPT_", "")
            if os.path.exists(os.path.join(SOURCE_FOLDER, file_name)):
                os.remove(os.path.join(SOURCE_FOLDER, file_name))